In [1]:
import numpy as np
import astropy.io.fits as fits
import pandas as pd
import math
pd.set_option('display.max_colwidth', None)

# Useful
# http://ps1images.stsci.edu/ps1_dr2_api.html

from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib
    
# My function: get ps1 object id from transient Ra/Dec
def get_ps1_dr2_objid_df_from_ra_dec(ztfid_l, ra_l, dec_l, df_name, radius=1.0/3600.0):
    if len(ztfid_l)>1:
        df_l = []
        for i, (ztfid, ra, dec) in enumerate(zip(ztfid_l, ra_l, dec_l)):
            
            if i % 1000 == 0: 
                print(f"i={i}: Saving!")
                if i != 0:
                    temp_df = pd.concat(df_l)
                    temp_df.to_csv(f'../dataframes/{df_name}_i={i}.csv') 
            
            ztfid = ztfid
            ra, dec = ra, dec
            radius = radius # radius = 1 arcsec
            constraints = {'nDetections.gt':1}

            # strip blanks and weed out blank and commented-out values
            columns = """objID,raMean,decMean,nDetections""".split(',')
            columns = [x.strip() for x in columns]
            columns = [x for x in columns if x and not x.startswith('#')]

            results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
            try:
                tab = ascii.read(results)
                tab = tab.to_pandas()
                tab['ztf_object_id'] = ztfid
                tab = tab.set_index('ztf_object_id')

            except: 
                print(f"Bad results for {ztfid}, ra={ra}, dec={dec}. Add dummy var. Continue!")
                print(results)
                
                temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
                tab = temp_df.set_index('ztf_object_id')
                
            df_l.append(tab)
        tab_df = pd.concat(df_l)
        
    else: # One entry
        ztfid = ztfid_l
        ra, dec = ra_l, dec_l
        radius = radius # radius = 1 arcsec
        constraints = {'nDetections.gt':1}

        # strip blanks and weed out blank and commented-out values
        columns = """objID,raMean,decMean,nDetections""".split(',')
        columns = [x.strip() for x in columns]
        columns = [x for x in columns if x and not x.startswith('#')]

        results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
        try:
            tab = ascii.read(results)
            tab = tab.to_pandas()
            tab['ztf_object_id'] = ztfid
            tab = tab.set_index('ztf_object_id')
        except: 
            print("Bad results. Add dummy var. Return!")
            print(results)
            
            temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), 
            columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
            tab = temp_df.set_index('ztf_object_id')

        tab_df = tab
    return tab_df


def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)


def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = f"{baseurl}/{release}/{table}.{format}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = f"{baseurl}/{release}/{table}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def mastQuery(request):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object

    Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position"""

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    headers,resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [2]:
with fits.open('../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits') as F:
    print(F.info())
    print(F[0].header.cards)
    objid=F[1].data['objid']
    ps_score=F[1].data['ps_score']

Filename: ../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1                1 BinTableHDU     14   16682163R x 2C   [K, E]   
None
('SIMPLE', True, 'conforms to FITS standard')
('BITPIX', 8, 'array data type')
('NAXIS', 0, 'number of array dimensions')
('EXTEND', True, '')
('HLSPTARG', '0', '')
('INSTRUME', 'PS1 survey', '')
('OBSERVAT', 'Haleakala Observatories', '')
('TELESCOP', 'Pan-STARRS1 Telescope', '')
('HLSPID', 'ps1_psc', '')
('HLSPLEAD', 'Adam A Miller', '')
('HLSPNAME', 'Probabilistic Classifications of Unresolved Point Sources in PanSTARRS1', '')
('HLSPVER', 'v1', '')
('LICENSE', 'CC BY 4.0', '')
('LICENURL', 'https://creativecommons.org/licenses/by/4.0/', '')
('REFERENC', 'https://doi.org/10.1088/1538-3873/aae3d9', '')
('HLSPDOI', 'https://doi.org/10.17909/t9-xjrf-7g34', '')


In [3]:
len(objid)

16682163

In [4]:
objid

array([108082954545197165, 108042899536471576, 108173097098982498, ...,
       108862811948172319, 109012792577601116, 108910340111860145])

In [5]:
ps_score[np.where(objid == 108082954545197165)[0][0]]

0.5813869

In [6]:
# score of 0 corresponding to extended objects and 1 corresponding to point sources
# extended objects are resolved galaxies
# point sources are unresolved stars or QSOs

In [7]:
len(ps_score)

16682163

In [8]:
ps_score

array([0.5813869, 0.8385   , 0.99875  , ..., 0.5611875, 0.9922917,
       0.       ], dtype=float32)

In [ ]:
%%time

# Instead of above, read in from csv files with host gal info!
# Visually vetted GHOST results split up over XXX files
# So merge them

dfs = []
for start_idx, end_idx in zip(range(0, 90100, 100), range(100, 90200, 100)): #90199, 90299
    df = pd.read_csv(f'./host_info/dataset_bank_hosts_start_idx:end_idx={start_idx}:{end_idx}.csv')
    dfs.append(df)
    
g_hosts_df = pd.concat(dfs, axis=0).reset_index(drop=True)
g_hosts_df.drop_duplicates(subset="TransientName", keep='first', inplace=True)
g_hosts_df = g_hosts_df.set_index('TransientName')
g_hosts_df.sort_values('TransientName', inplace=True)
#g_hosts_df.reset_index(inplace=True, drop=True)
g_hosts_df

In [ ]:
g_hosts_has_ps1_objid_df = g_hosts_df[~g_hosts_df.objID.isna()]
g_hosts_has_ps1_objid_df.index.rename("ztf_object_id", inplace=True)
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df[g_hosts_has_ps1_objid_df.index.str.startswith('ZTF')]
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df[g_hosts_has_ps1_objid_df.index.str.contains("\n")==False]
g_hosts_has_ps1_objid_df.to_csv('../dataframes/g_hosts_has_ps1_objid_df.csv')
g_hosts_has_ps1_objid_df

# TRANSIENT (Not Host) ps1 objIDs

In [ ]:
%%time
transient_df_name = 'get_ps1_dr2_objid_transient_coords_df'
hosts_df_name = 'get_ps1_dr2_objid_ghost_hosts_assoc_coords_df'

get_ps1_dr2_objid_transient_coords_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['TransientRA'],
                                 dec_l=g_hosts_has_ps1_objid_df['TransientDEC'],
                                 df_name=transient_df_name)
get_ps1_dr2_objid_transient_coords_df.to_csv(f'../dataframes/{transient_df_name}.csv')
get_ps1_dr2_objid_transient_coords_df

# TODO: HOST (not transient) ps1 objIDs

In [ ]:
%%time
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['raMean'],
                                 dec_l=g_hosts_has_ps1_objid_df['decMean'],
                                 df_name=hosts_df_name)
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df.to_csv(f'../dataframes/{hosts_df_name}.csv')
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df

In [ ]:
# These ZTF18aalsfbd, ZTF18aalroha, ZTF18aaacqmh are transients at start of ztf survey...
# Others like ZTF17aabcirl, ZTF17aabfgmo, ZTF17aabvisz, ZTF17aadnacr are stars...

In [13]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aaaessf'], ra_l=6.99479767152418, dec_l=34.47048854338335, df_name="blah", radius=9.0/3600.0)

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaessf,149360069947635231,6.994798,34.470489,59


In [51]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF18aalroha'], ra_l=217.0604874344824, dec_l=40.866147144851006, radius=2.0/3600.0)

Bad results. See output. Then skip!



In [53]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aabcirl'], ra_l=19.51308273339673, dec_l=23.40903684468689, radius=1.50/3600.0)

Bad results. See output. Then skip!



In [34]:
df_ah = pd.read_csv(f'../dataframes/get_ps1_dr2_objid_transient_coords_df_i=200.csv')
df_ah

,ztf_object_id,objID,raMean,decMean,nDetections
0,ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61
1,ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64
2,ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57
3,ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77
4,ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77
...,...,...,...,...,...
207,ZTF17aabqajc,157260061006557656,6.100616,41.055845,63
208,ZTF17aabrisn,113130319147436034,31.914750,4.279631,70
209,ZTF17aabsamc,96890801274185678,80.127441,-9.253931,85
210,ZTF17aabslfb,163651087504864995,108.750441,46.378624,2


In [20]:
test2 = g_hosts_has_ps1_objid_df
test2 = test2[test2.index.str.contains("\n")==False]
g_hosts_has_ps1_objid_df.iloc[19:25]

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaessf,PSO J002758.152+342737.379,NaN,NaN,NaN,149350069924292964,910743000025002,198689482105259,0,17242199554318,14935,...,1.0,8.50073,3.59930,1.34110,1.30147,6.296310,NaN,0.0,False,NaN
ZTF17aaaessf\nZTF17aaaessf,PSO J002758.152+342737.379,NaN,NaN,NaN,149350069924292964,910743000025002,198689482105259,0,17242199554318,14935,...,1.0,8.50073,3.59930,1.34110,1.30147,6.296310,NaN,0.0,False,NaN
ZTF17aaaeszu,PSO J002725.144+410817.821,NaN,NaN,NaN,157360068547966611,910400000060829,234397840240029,0,17318542320870,15736,...,1.0,8.56861,3.88292,5.23500,4.23818,3.732670,NaN,0.0,False,NaN
ZTF17aaafssi,PSO J012811.769+304923.587,NaN,NaN,NaN,144980220490828453,924746000021116,200489073398395,0,17348710249472,14498,...,1.0,11.74970,3.79276,1.34208,1.21408,1.012490,NaN,0.0,False,NaN
ZTF17aaafstx,PSO J012907.611+303613.108,NaN,NaN,NaN,144720222814224929,924746000062710,200489073439989,0,17348703370033,14472,...,1.0,7.89906,3.50652,2.62227,3.55783,NaN,NaN,0.0,False,NaN
ZTF17aaafvft,PSO J021026.532-101523.141,NaN,NaN,NaN,95690326106512679,930138000016982,404238026949205,0,8992209949747,9569,...,1.0,7.73635,1.79648,3.03877,1.37975,0.611368,NaN,0.0,False,NaN


# TODO: match PS1 object to transient ra/dec, then apply T&M ps_score

In [11]:
#g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df.head()

In [12]:
g_hosts_has_ps1_objid_df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaaenu,PSO J014921.296+353104.780,NaN,NaN,NaN,150620273387282157,942081000036636,201335181971228,0,17342042780099,15062,...,1.0,13.51850,4.98918,4.267330,3.78907,9.400590,NaN,0.0,False,NaN
ZTF17aaaazhi,PSO J021351.579+333011.621,NaN,NaN,NaN,148200334649204503,936092000041298,201897822691666,0,17567891492385,14820,...,1.0,5.74010,1.27822,0.996905,1.49481,0.999151,NaN,0.0,False,NaN
ZTF17aaaazmz,PSO J020108.703+354438.836,NaN,NaN,NaN,150890302862623476,944990000044342,201356656815413,0,17342424965577,15089,...,1.0,4.73348,2.72390,3.379060,3.34216,4.355060,NaN,0.0,False,NaN
ZTF17aaabghb,PSO J050857.900-010153.780,NaN,NaN,NaN,106760772412722499,969245000054665,368151711765898,0,9372013840704,10676,...,1.0,5.70637,5.27978,7.873000,3.75759,2.645650,NaN,0.0,False,NaN
ZTF17aaabuab,PSO J032818.391-021919.410,NaN,NaN,NaN,105210520766363928,947850000033169,365261198754193,0,9429305171894,10521,...,1.0,3.08845,4.04116,4.027430,3.76517,11.518300,NaN,0.0,False,NaN


In [13]:
%%time
for ztfid, ra, dec, ps1OID in zip(g_hosts_has_ps1_objid_df.index,
                           g_hosts_has_ps1_objid_df['raMean'], 
                           g_hosts_has_ps1_objid_df['decMean'], 
                           g_hosts_has_ps1_objid_df['objID']):    
    
    
    if dec >= 0: dec_int = int(math.floor(dec))
    else: dec_int = int(math.ceil(dec))
    
    # Open fits file corresponding to declination
    with fits.open(f'../ps1_psc/hlsp_ps1-psc_ps1_gpc1_{dec_int}_multi_v1_cat.fits') as F:
        objid_arr=F[1].data['objid']
        ps_score_arr=F[1].data['ps_score']
        
        try:
            ps_score = ps_score_arr[np.where(objid_arr == ps1OID)[0][0]]
            print(ps1OID, ps_score)
        except:
            print(f"{ps1OID} is not in this file. Set ps_score to -99")
            ps_score = -99

150620273387282157 0.025833333
148200334649204503 0.9295833
150890302862623476 0.0
106760772412722499 0.00375
105210520766363928 0.016416667
CPU times: user 366 ms, sys: 563 ms, total: 929 ms
Wall time: 3.67 s


In [15]:
for ztfid, (ra_t, dec_t, ra_host, dec_host) in g_hosts_has_ps1_objid_df[['TransientRA', 'TransientDEC', 'raMean', 'decMean']].iterrows():
    print(ztfid)
    print(f'Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_t}+{dec_t}&filter=color')
    #ps1_dr2_objid = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=[ztfid], ra_l=ra_t, dec_l=dec_t)
    #print(ps1_dr2_objid)
    print(f'Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_host}+{dec_host}&filter=color')
    #ps1_dr2_objid = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=[ztfid], ra_l=ra_host, dec_l=dec_host)
    #print(ps1_dr2_objid)
    print(f'\n')


ZTF17aaaaenu
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.35194614323238+35.51434387930057&filter=color
                            objID     raMean    decMean  nDetections
ztf_object_id                                                       
ZTF17aaaaenu   150610273519657838  27.351981  35.514352           61
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.33876949+35.5179556&filter=color
                            objID     raMean    decMean  nDetections
ztf_object_id                                                       
ZTF17aaaaenu   150620273387282157  27.338759  35.517955           38


ZTF17aaaazhi
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=33.467575772476465+33.4979008400576&filter=color
                            objID     raMean   decMean  nDetections
ztf_object_id                                                      
ZTF17aaaazhi   148190334676288060  33.467597  33.49786           64
Host: http://ps1images.stsci.edu/cgi-bin/p

ValueError: Length of values (1) does not match length of index (2)

In [16]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aaaaenu'], ra_l=27.33876949, dec_l=35.5179556)

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38


In [18]:
get_ps1_dr2_objid_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['TransientRA'],
                                 dec_l=g_hosts_has_ps1_objid_df['TransientDEC'])
get_ps1_dr2_objid_df.to_csv('../dataframes/get_ps1_dr2_objid_df.csv')
get_ps1_dr2_objid_df

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61
ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64
ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57
ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77
ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77
